# Predicting Hand-written Digits on the MNIST Dataset

The MNIST dataset is a well-known, massive dataset of images of handwritten digits, 0 - 9. Thanks to its size, it's a common dataset in use for classification tasks. Our goal is to construct a classifier that, given an image of a handwritten digit, it predicts which digit it is.

Here, we'll be trying to learn a good classifier using two different algorithms, Adagrad and ADAM.

## Loading the MNIST dataset, loss and gradient

In [2]:
from keras.datasets import mnist
import numpy as np

# Loading the data set
(train_X, train_y), (test_X, test_y) = mnist.load_data()

# For debugging purposes, you might want to restrict the number of labels
# you are learning (e.g., you could begin with k=2).
# However, in the end we want to have a classifier for all 10 digits.
num_classes = 10

# Filtering the data set to include only k labels
train_X = train_X[train_y < num_classes]
train_y = train_y[train_y < num_classes]
test_X = test_X[test_y < num_classes]
test_y = test_y[test_y < num_classes]

# Collapsing the 28 x 28 feature matrix into a length 784 feature vector
train_X = train_X.reshape(train_X.shape[0], train_X.shape[1] * train_X.shape[2]).astype(float)
test_X = test_X.reshape(test_X.shape[0], test_X.shape[1] * test_X.shape[2]).astype(float)

# Subtracting the mean and normalizing
mean = np.mean(train_X, axis=0)
print(mean.shape)
train_X -= mean
test_X -= mean
std = np.std(train_X, axis=0)
std = np.where(std == 0, 1, std)
train_X /= std
test_X /= std

# Adding the all ones vector in place of a bias term
train_X = np.hstack((train_X, np.ones((len(train_X), 1))))
test_X = np.hstack((test_X, np.ones((len(test_X), 1))))

(784,)


In [3]:
def nn_predict(X, W1, b1, W2, b2):
    hidden_layer = np.maximum(0, np.dot(X, W1.T) + b1) # ReLU activation
    scores = np.dot(hidden_layer, W2.T) + b2
    y_pred = np.argmax(scores, axis=1)
    return y_pred

def evaluate_accuracy(X, y, W1, b1, W2, b2):
    y_pred = nn_predict(X, W1, b1, W2, b2)
    accuracy = np.mean(y_pred == y)
    return accuracy

def softmax(x):
    x = x - np.max(x, axis=1, keepdims=True)
    return np.exp(x) / np.sum(np.exp(x), axis=1, keepdims=True)

def nn_loss_grad(X, y, W1, b1, W2, b2, reg):
    """
    Compute the loss and gradients of the neural network.
    Feature vectors have dimension n, there are K classes, and m training examples.

    Inputs:
    - W: A numpy array of shape (K, n) containing weights.
    - X: A numpy array of shape (m, n) containing the training feature vectors 
        (row i of X is the feature vector of the i-th training example).
    - y: A numpy array of shape (m,) containing the training labels; y[i] = c means
      that X[i] has label c, where 0 <= c < K.
    - reg: (float) regularization strength (the gamma value from lecture)

    Returns a tuple of:
    - loss as single float
    - gradient with respect to weights W1; an array of same shape as W1
    - gradient with respect to the offsets b1; an array of the same shape as b1
    - gradient with respect to weights W2; an array of same shape as W2
    - gradient with respect to the offsets b2; an array of the same shape as b2
    """

    num_examples, _ = X.shape

    hidden_layer = np.maximum(0, np.dot(X, W1.T) + b1) # ReLU activation
    scores = np.dot(hidden_layer, W2.T) + b2
    
    # compute the class probabilities
    probs = softmax(scores)

    # compute the loss: average cross-entropy loss and regularization
    corect_logprobs = -np.log(probs[range(num_examples),y] + 1e-10)
    data_loss = np.sum(corect_logprobs)/num_examples
    reg_loss = 0.5*reg*np.sum(W1*W1) + 0.5*reg*np.sum(W2*W2)
    loss = data_loss + reg_loss

    # compute the gradient on scores
    dscores = probs
    dscores[range(num_examples),y] -= 1
    dscores /= num_examples

    # backpropate the gradient to the parameters
    # first backprop into parameters W2 and b2
    dW2 = np.dot(hidden_layer.T, dscores).T
    db2 = np.sum(dscores, axis=0)
    # next backprop into hidden layer
    dhidden = np.dot(dscores, W2)
    # backprop the ReLU non-linearity
    dhidden[hidden_layer <= 0] = 0
    # finally into W1,b1
    dW1 = np.dot(X.T, dhidden).T
    db1 = np.sum(dhidden, axis=0)

    # add regularization gradient contribution
    dW2 += reg * W2
    dW1 += reg * W1

    return loss, dW1, db1, dW2, db2

# Adagrad

In [4]:

def nn_train_adagrad(X, y, hidden_layer_size=100, reg=1e-3, num_iter=100, verbose=False):
     """
     Train an neural network with a single hidden layer using  adagrad.

     Inputs:
     - X: A numpy array of shape (m, n) containing the training feature vectors 
          (row i of X is the feature vector of the i-th training example).
     - y: A numpy array of shape (m,) containing the training labels; y[i] = c means
          that X[i] has label c, where 0 <= c < K.
     - hidden_layer_size: (int) size of the hidden layer.
     - gamma: (float) regularization strength.
     - num_iters: (integer) number of gradient descent iterations.
     - verbose: (boolean) If true, print progress during optimization.

     Outputs:
     - W1 : matrix of weights of the first layer
     - b1 : vector of offsets of the first layer
     - W1 : matrix of weights of the output layer
     - b1 : vector of offsets of the output layer
     """

     _, num_features = X.shape

     # initialize parameters randomly
     W1 = 0.01 * np.random.randn(hidden_layer_size, num_features)
     b1 = np.zeros(hidden_layer_size)
     W2 = 0.01 * np.random.randn(num_classes, hidden_layer_size)
     b2 = np.zeros(num_classes)

     #storage
     cache_W1 = np.zeros_like(W1)
     cache_b1 = np.zeros_like(b1)
     cache_W2 = np.zeros_like(W2)
     cache_b2 = np.zeros_like(b2)

     for i in range(num_iter):   
          loss, dW1, db1, dW2, db2 = nn_loss_grad(X, y, W1, b1, W2, b2, reg)
          cache_W1 += dW1 ** 2
          cache_b1 += db1 ** 2
          cache_W2 += dW2 ** 2
          cache_b2 += db2 ** 2
          W1 -= dW1 / (np.sqrt(cache_W1 + 1e-10) )
          b1 -= db1 / (np.sqrt(cache_b1 + 1e-10) )
          W2 -= dW2 / (np.sqrt(cache_W2 + 1e-10) )
          b2 -= db2 / (np.sqrt(cache_b2 + 1e-10) )
          if verbose and i % 10 == 0:
               print("iteration %d: loss %f" % (i, loss))

     return W1, b1, W2, b2

In [5]:
#predict using Adagrad
W1, b1, W2, b2 = nn_train_adagrad(train_X, train_y, hidden_layer_size=100, reg=1e-3, num_iter=100, verbose=True)
train_accuracy = evaluate_accuracy(train_X, train_y, W1, b1, W2, b2)
test_accuracy = evaluate_accuracy(test_X, test_y, W1, b1, W2, b2)
print("Train accuracy: ", train_accuracy)
print("Test accuracy: ", test_accuracy)


iteration 0: loss 2.304814
iteration 10: loss 35.793722
iteration 20: loss 26.589574
iteration 30: loss 21.391386
iteration 40: loss 17.826571
iteration 50: loss 15.211064
iteration 60: loss 13.012247
iteration 70: loss 11.420146
iteration 80: loss 10.036550
iteration 90: loss 8.916381
Train accuracy:  0.9753333333333334
Test accuracy:  0.9475


# ADAM

In [6]:
def nn_train_adam(X, y, hidden_layer_size=100, reg=1e-3, num_iter=100, verbose=False):
     """
     Train an neural network with a single hidden layer using  ADAM.

     Inputs:
     - X: A numpy array of shape (m, n) containing the training feature vectors 
          (row i of X is the feature vector of the i-th training example).
     - y: A numpy array of shape (m,) containing the training labels; y[i] = c means
          that X[i] has label c, where 0 <= c < K.
     - hidden_layer_size: (int) size of the hidden layer.
     - gamma: (float) regularization strength.
     - num_iters: (integer) number of gradient descent iterations.
     - verbose: (boolean) If true, print progress during optimization.

     Outputs:
     - W1 : matrix of weights of the first layer
     - b1 : vector of offsets of the first layer
     - W1 : matrix of weights of the output layer
     - b1 : vector of offsets of the output layer
     """

     _, num_features = X.shape

     # initialize parameters randomly
     W1 = 0.01 * np.random.randn(hidden_layer_size, num_features)
     b1 = np.zeros(hidden_layer_size)
     W2 = 0.01 * np.random.randn(num_classes, hidden_layer_size)
     b2 = np.zeros(num_classes)
     beta1 = 0.9
     beta2 = 0.999

     #storage
     m_W1 = np.zeros_like(W1)
     m_b1 = np.zeros_like(b1)
     m_W2 = np.zeros_like(W2)
     m_b2 = np.zeros_like(b2)
     v_W1 = np.zeros_like(W1)
     v_b1 = np.zeros_like(b1)
     v_W2 = np.zeros_like(W2)
     v_b2 = np.zeros_like(b2)
     
     for i in range(num_iter):
          loss, dW1, db1, dW2, db2 = nn_loss_grad(X, y, W1, b1, W2, b2, reg)
          m_W1 = beta1 * m_W1 + (1 - beta1) * dW1
          m_b1 = beta1 * m_b1 + (1 - beta1) * db1
          m_W2 = beta1 * m_W2 + (1 - beta1) * dW2
          m_b2 = beta1 * m_b2 + (1 - beta1) * db2
          v_W1 = beta2 * v_W1 + (1 - beta2) * dW1 ** 2
          v_b1 = beta2 * v_b1 + (1 - beta2) * db1 ** 2
          v_W2 = beta2 * v_W2 + (1 - beta2) * dW2 ** 2
          v_b2 = beta2 * v_b2 + (1 - beta2) * db2 ** 2
          W1 = W1 - m_W1 / (np.sqrt(v_W1) + 1e-10)
          b1 = b1 - m_b1 / (np.sqrt(v_b1) + 1e-10)
          W2 = W2 - m_W2 / (np.sqrt(v_W2) + 1e-10)
          b2 = b2 - m_b2 / (np.sqrt(v_b2) + 1e-10)
          if verbose and i % 10 == 0:
               print("iteration %d: loss %f" % (i, loss))

     return W1, b1, W2, b2

In [7]:
#predict using Adam
W1, b1, W2, b2 = nn_train_adam(train_X, train_y, hidden_layer_size=100, reg=1e-3, num_iter=100, verbose=True)
train_accuracy = evaluate_accuracy(train_X, train_y, W1, b1, W2, b2)
test_accuracy = evaluate_accuracy(test_X, test_y, W1, b1, W2, b2)
print("Train accuracy: ", train_accuracy)
print("Test accuracy: ", test_accuracy)


iteration 0: loss 2.305880
iteration 10: loss 4445.414071
iteration 20: loss 6531.377248
iteration 30: loss 4932.266780
iteration 40: loss 2938.705231
iteration 50: loss 1639.890035
iteration 60: loss 935.877995
iteration 70: loss 577.587407
iteration 80: loss 386.925530
iteration 90: loss 287.989865
Train accuracy:  0.9730166666666666
Test accuracy:  0.9385


# ADAM with Stochastic Gradient Descent

In [16]:
def nn_train_adam_stoch(X, y, hidden_layer_size=100, reg=1e-3, num_stoch=10, num_iter=1000000, verbose=False):
     """
     Train an neural network with a single hidden layer using  ADAM and stochastic evaluations
     of the gradient.

     Inputs:
     - X: A numpy array of shape (m, n) containing the training feature vectors 
          (row i of X is the feature vector of the i-th training example).
     - y: A numpy array of shape (m,) containing the training labels; y[i] = c means
          that X[i] has label c, where 0 <= c < K.
     - hidden_layer_size: (int) size of the hidden layer.
     - reg: (float) regularization strength.
     - num_stoch: (integer) number of training examples used for evaluating the
          stochastic gradient
     - num_iters: (integer) number of gradient descent iterations.
     - verbose: (boolean) If true, print progress during optimization.

     Outputs:
     - W1 : matrix of weights of the first layer
     - b1 : vector of offsets of the first layer
     - W1 : matrix of weights of the output layer
     - b1 : vector of offsets of the output layer
     """

     _, num_features = X.shape

     # initialize parameters randomly
     W1 = 0.01 * np.random.randn(hidden_layer_size, num_features)
     b1 = np.zeros(hidden_layer_size)
     W2 = 0.01 * np.random.randn(num_classes, hidden_layer_size)
     b2 = np.zeros(num_classes)
     beta1 = 0.9
     beta2 = 0.999

     #storage
     m_W1 = np.zeros_like(W1)
     m_b1 = np.zeros_like(b1)
     m_W2 = np.zeros_like(W2)
     m_b2 = np.zeros_like(b2)
     v_W1 = np.zeros_like(W1)
     v_b1 = np.zeros_like(b1)
     v_W2 = np.zeros_like(W2)
     v_b2 = np.zeros_like(b2)
     
     for i in range(num_iter):
          index = np.random.choice(X.shape[0], num_stoch, replace=False)
          X_stoch = X[index]
          y_stoch = y[index]
          loss, dW1, db1, dW2, db2 = nn_loss_grad(X_stoch, y_stoch, W1, b1, W2, b2, reg)
          m_W1 = beta1 * m_W1 + (1 - beta1) * dW1
          m_b1 = beta1 * m_b1 + (1 - beta1) * db1
          m_W2 = beta1 * m_W2 + (1 - beta1) * dW2
          m_b2 = beta1 * m_b2 + (1 - beta1) * db2
          v_W1 = beta2 * v_W1 + (1 - beta2) * dW1 ** 2
          v_b1 = beta2 * v_b1 + (1 - beta2) * db1 ** 2
          v_W2 = beta2 * v_W2 + (1 - beta2) * dW2 ** 2
          v_b2 = beta2 * v_b2 + (1 - beta2) * db2 ** 2
          W1 = W1 - m_W1 / (np.sqrt(v_W1) + 1e-10)
          b1 = b1 - m_b1 / (np.sqrt(v_b1) + 1e-10)
          W2 = W2 - m_W2 / (np.sqrt(v_W2) + 1e-10)
          b2 = b2 - m_b2 / (np.sqrt(v_b2) + 1e-10)
          if verbose and i % 100 == 0:
               print("iteration %d: loss %f" % (i, loss))

     return W1, b1, W2, b2

In [18]:
#predict using Adam-SGD
W1, b1, W2, b2 = nn_train_adam_stoch(train_X, train_y, hidden_layer_size=250, reg=1e-3, num_iter=10000, verbose=True)
train_accuracy = evaluate_accuracy(train_X, train_y, W1, b1, W2, b2)
test_accuracy = evaluate_accuracy(test_X, test_y, W1, b1, W2, b2)
print("Train accuracy: ", train_accuracy)
print("Test accuracy: ", test_accuracy)


iteration 0: loss 2.317678
iteration 100: loss 174332.270786
iteration 200: loss 158407.492954
iteration 300: loss 148134.371158
iteration 400: loss 138888.443435
iteration 500: loss 127680.454754
iteration 600: loss 117720.739075
iteration 700: loss 108337.389703
iteration 800: loss 102850.099982
iteration 900: loss 96426.775901
iteration 1000: loss 90835.701148
iteration 1100: loss 85982.654150
iteration 1200: loss 80316.471789
iteration 1300: loss 75041.738042
iteration 1400: loss 69452.834070
iteration 1500: loss 65783.347702
iteration 1600: loss 62116.760334
iteration 1700: loss 59293.642066
iteration 1800: loss 55588.731878
iteration 1900: loss 52156.664698
iteration 2000: loss 50152.572781
iteration 2100: loss 49009.163089
iteration 2200: loss 45989.530143
iteration 2300: loss 46762.389024
iteration 2400: loss 44776.709190
iteration 2500: loss 42499.679972
iteration 2600: loss 40513.380763
iteration 2700: loss 38185.285446
iteration 2800: loss 35981.930684
iteration 2900: loss 3